# ResNet Training

В этом ноутбуке будет произведено обучение модели `Resnet34` на полученном наборе данных из ноутбука
`./notebooks/01. get_train_data`.

In [1]:
import os
import sys
import numpy as np
import random
from tqdm import tqdm

sys.path.append('.')
from definitions import ROOT_DIR
from detectime.utils import (
    save_checkpoint,
    train_valid_split
)
from detectime.models import load_model
from detectime.loss_function import get_loss
from detectime.optimizers import (
    get_optimizer,
    get_scheduler
)
from detectime.dataset import get_data_loaders
from detectime.train import train, validation

%matplotlib inline
%load_ext autoreload
%autoreload 2

{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "matplotlib data path: C:\\Users\\shiro\\Desktop\\detectime\\lib\\site-packages\\matplotlib\\mpl-data"}
{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "CONFIGDIR=C:\\Users\\shiro\\.matplotlib"}
{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "matplotlib version 3.4.2"}
{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "interactive is False"}
{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "platform is win32"}
{"asctime": "2021-07-03 05:14:51", "name": "matplotlib", "filename": "__init__.py", "levelname": "DEBUG", "message": "loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '

In [2]:
DATA_PATH = ROOT_DIR / 'data'
NOTEBOOK_PATH = ROOT_DIR / 'notebooks'
INPUT_DATA = DATA_PATH / 'INPUT_DATA'
INPUT_IMAGES_FOLDER = INPUT_DATA / 'TRAIN_DATA'
TRAIN_IMG_FOLDER = INPUT_DATA / 'TRAIN_IMG'
SAVE_TRAIN_IMAGES_HANDS = TRAIN_IMG_FOLDER / 'HANDS'
SAVE_TRAIN_IMAGES_FACES= TRAIN_IMG_FOLDER / 'FACES'

JSON_FOLDER = INPUT_DATA / 'JSON'
FACES_JSON_PRETRAINED = JSON_FOLDER / 'train_with_bboxes.json'
TRAIN_LABELS = INPUT_DATA / 'train.csv'
HAND_DETECTION_FOLDER = ROOT_DIR / 'model' / 'mask_rcnn_hand_detection.h5'
ANNOTATION_DATA = JSON_FOLDER / 'hands.json'

Подгрузим известный нам конфиг и будем использовать `device` для обучения - либо `cpu`, либо `cuda`.

In [3]:
import torch
import yaml
from detectime.utils import convert_dict_to_tuple

CONFIG_PATH = ROOT_DIR / 'config.yml'

with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cpu


Обеспечим консистентность и воспроизводимость системы.

In [4]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

Подгрузим веса модели `ResNet34`.

In [5]:
print("Loading model...")
net = load_model(config, device=device_name)
print("Done.")

Loading model...
{"asctime": "2021-07-03 05:14:52", "name": "detectime.models", "filename": "models.py", "levelname": "INFO", "message": "ResNet34"}
Done.


Подгрузим optimizer, scheduler и loss aggregator.

In [6]:
criterion, criterion_val = get_loss(config, device=device_name)

In [12]:
optimizer = get_optimizer(config, net)

{"asctime": "2021-07-03 05:15:47", "name": "detectime.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "0.002"}
{"asctime": "2021-07-03 05:15:47", "name": "detectime.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "Opt: SGD"}


In [8]:
n_epoch = config.train.n_epoch
scheduler = get_scheduler(config, optimizer)
train_epoch = tqdm(range(config.train.n_epoch),
                   dynamic_ncols=True,
                   desc='Epochs',
                   position=0)

Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

Разделим данные на обучение и валидацию по ключу `video_name`.

In [9]:
train_data, val_data = train_valid_split(
    ANNOTATION_DATA,
    JSON_FOLDER,
    val_size=0.4
)

{"asctime": "2021-07-03 05:14:55", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "55"}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "3"}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "train data length=47"}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "validation data length=8"}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "Savedir train and valid data: C:\\Users\\shiro\\Desktop\\new\\detectime\\data\\INPUT_DATA\\JSON"}


In [10]:
ANNOTATIONS_TRAIN = JSON_FOLDER / 'train.json'
ANNOTATIONS_VALID= JSON_FOLDER / 'valid.json'

In [11]:
dt, dv = get_data_loaders(config,
                          INPUT_IMAGES_FOLDER,
                          ANNOTATIONS_TRAIN,
                          ANNOTATIONS_VALID
                          )

out_dir = str(DATA_PATH / os.path.join(config.outdir, 'test'))
print("Savedir: {}".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

{"asctime": "2021-07-03 05:14:55", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing train reader..."}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing valid reader..."}
{"asctime": "2021-07-03 05:14:55", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
Savedir: C:\Users\shiro\Desktop\new\detectime\data\experiments\test


C:\Users\shiro\Desktop\detectime\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
for epoch in train_epoch:
    train(net, dt, criterion, optimizer, config, epoch)
    validation(net, dv, criterion_val, epoch)
    save_checkpoint(net, optimizer, scheduler, epoch, out_dir)
    scheduler.step()